In [1]:
# pip install awswrangler

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import awswrangler as wr

sys.path.append(os.path.dirname(os.getcwd())+'/src')
from preprocess import preprocess_module

In [3]:
preproc = preprocess_module()
print('Loaded preproc module')

df_fragments = preproc.get_fragments()
text = df_fragments[['Texto Limpo']].drop_duplicates()
text = preproc.preprocess_fragments(text, 'Texto Limpo')
print('Preprocessed fragments')

Loaded preproc module
Preprocessed fragments


In [4]:
text = pd.concat([text[['processed']].iloc[0:500].copy(), text[['processed']].iloc[1000:1500].copy()], axis=0)
text = preproc.find_inequality(df=text, col='processed')
print('Found power plant in fragments\' text')

Found power plant in fragments' text


In [5]:
text = text[text['INEQUACAO'].astype(bool)]

In [6]:
len(text)

90

In [7]:
text

,processed,INEQUACAO
48,1 . Energização iniciando pela SE Peixe 2 ( s...,"[Tensão na SE Peixe 2 ≤ 545 kV, SE Peixe 2 ≤ 5..."
49,PassoCoordenaçãoControleComando / ExecuçãoProc...,[ V ≤ 15 kV]
50,1 . Energização iniciando pela SE Serra da Me...,"[SE Serra da Mesa ≤ 540 kV, SE Serra da Mesa ≤..."
52,1 . Energização iniciando pela SE Serra da Me...,"[Tensão na SE Serra da Mesa 2 ≤ 545 kV, SE Pei..."
53,1 . Energização iniciando pela SE Serra da Me...,"[Tensão na SE Serra da Mesa 2 ≤ 550 kV, SE Ser..."
...,...,...
422,1 . Desenergização iniciando pela SE Biguaçu ...,"[SE Gaspar 2 inferior a 239 kV, tensão na SE G..."
423,1 . Energização iniciando pela SE Gaspar 2 ( ...,[Tensão na SE Biguaçuigual ou inferior a 240 k...
426,1 . Energização iniciando pela SE Biguaçu ( S...,"[ δ ≤ 31 °, Tensão na SE Biguaçuigual ou infer..."
429,1 . Energização iniciando pela SE Biguaçu ( S...,"[SE Palhoçaigual ou inferior a 242 kV, Tensão ..."


In [8]:
samples = preproc.create_synthetic_samples(
    text,
    entity_col='INEQUACAO',
    text_col='processed'
    )
samples.drop_duplicates(inplace=True)
print('Created samples dataframe for power plant entity')

Created samples dataframe for power plant entity


In [9]:
bucket = 'ons-ds-dev-entityrecognition'
samples_path = f's3://{bucket}/mpo/mvp/data/processed/inequality_samples.parquet'
samples.to_parquet(samples_path, compression='gzip')
print(f'saved samples in {samples_path}')

saved samples in s3://ons-ds-dev-entityrecognition/mpo/mvp/data/processed/inequality_samples.parquet


In [5]:
df = wr.s3.read_parquet('s3://ons-ds-dev-entityrecognition/mpo/mvp/data/raw/marked_samples/poc_marked.parquet')
df.head()

,region,text,start,end,label
0,cosr-nco,PassoCoordenaçãoControleComando / ExecuçãoProc...,97,106,ACAO_A_EXECUTAR
1,cosr-nco,PassoCoordenaçãoControleComando / ExecuçãoProc...,270,279,ACAO_A_EXECUTAR
2,cosr-nco,PassoCoordenaçãoControleComando / ExecuçãoProc...,373,381,ACAO_A_EXECUTAR
3,cosr-nco,PassoCoordenaçãoControleComando / ExecuçãoProc...,460,474,ACAO_A_EXECUTAR
4,cosr-nco,PassoCoordenaçãoControleComando / ExecuçãoProc...,493,502,ACAO_A_EXECUTAR


In [144]:
pos = 2170
start = df[df['label']=='INEQUACAO']['start'].iloc[pos]
text = df[df['label']=='INEQUACAO']['text'].iloc[pos]
text[start-2:]

' Tensão da SE que inicia a manobra inferior ou igual a 242 kV.2COSR-NECOSR-NEChesf (Cros)Caso necessário, solicitar à Chesf contatar com o consumidor FERBASA informando a normalização da LT. - '

In [55]:
import re
re.search(
    'V\(\w+\)',
    'V(GVAL) ≤ 240 kV'
)[0]

'V(GVAL)'

In [31]:
df

,text,entity,label,start,end
682,1 . Desenergização ( Ambos os Sentidos ) Pass...,UHE Boa Esperan,USINA,538,553
698,1 . Desenergização INICIANDO PELO TERMINAL DE...,UFV SJP,USINA,249,256
701,1 . Desenergização INICIANDO PELO TERMINAL DE...,Usinas conectadas,USINA,259,276
700,1 . Desenergização INICIANDO PELO TERMINAL DE...,UFV SJP fora de,USINA,282,297
718,1 . Desenergização iniciando pela SE Itajubá ...,usinas definidas na tabela,USINA,801,827
...,...,...,...,...,...
596,Os valores de FIV e FSE especificados são igua...,UHE Itaipu,USINA,417,427
568,"PassoCoordenaçãoControleComando , ExecuçãoProc...",UHE Serra da Mesa,USINA,601,618
569,"PassoCoordenaçãoControleComando , ExecuçãoProc...",UHE Serra da Mesa,USINA,753,770
570,"PassoCoordenaçãoControleComando , ExecuçãoProc...",UHE Serra da Mesa,USINA,825,842


In [30]:
len(set(df['text']))

71

In [21]:
df_a.sort_values(by=['start','end'], axis=0, ascending=True, inplace=True)
df_a.drop_duplicates(subset=['text','label','start'], keep='last', inplace=True)
df_a

,text,entity,label,start,end
1046,1 . Desenergização iniciando pela SE Biguaçu ...,inferior a,OPERADOR_MATEMATICO,280,290
1213,1 . Desenergização iniciando pela SE Biguaçu ...,inferior a,OPERADOR_MATEMATICO,283,293
1062,1 . Desenergização iniciando pela SE Biguaçu ...,inferior a,OPERADOR_MATEMATICO,287,297
294,1 . Desenergização PELO TERMINAL DE 230 KV ( ...,Inferior a,OPERADOR_MATEMATICO,398,408
291,1 . Desenergização PELO TERMINAL DE 230 KV ( ...,Entre,OPERADOR_MATEMATICO,414,419
296,1 . Desenergização PELO TERMINAL DE 230 KV ( ...,Superior a,OPERADOR_MATEMATICO,434,444
373,1 . Desenergização iniciando pela SE Cachoeir...,≤,OPERADOR_MATEMATICO,435,436
292,1 . Desenergização PELO TERMINAL DE 230 KV ( ...,Entre,OPERADOR_MATEMATICO,450,455
188,1 . Desenergização ( Ambos os Sentidos ) Pass...,No mínimo,OPERADOR_MATEMATICO,494,503
189,1 . Desenergização ( Ambos os Sentidos ) Pass...,entre,OPERADOR_MATEMATICO,638,643


In [3]:
preproc = preprocess_module()

In [4]:
df_fragments = preproc.get_fragments()
text = df_fragments[['Texto Limpo']].drop_duplicates()
text = preproc.preprocess_fragments(text, 'Texto Limpo')
print('Preprocessed fragments')

Preprocessed fragments


In [5]:
df_manualactions = preproc.get_manual_actions(
    bucket='ons-ds-dev-entityrecognition',
    path='mpo/mvp/data/raw/manual_actions'
    )
df_manualactions = preproc.preprocess_fragments(df_manualactions, 'actions')
print('Preprocessed manual actions')

Preprocessed manual actions


In [6]:
# df_manualactions.head(60)

In [7]:
text = text[['processed']].copy()
text = preproc.find_manual_actions(
    text,
    df_manualactions
    )
print('Found manual actions in fragments\' text')

Found manual actions in fragments' text


In [8]:
samples = preproc.create_synthetic_samples(
    text,
    entity_col='ACAO_MANUAL',
    text_col='processed'
    )
samples.drop_duplicates(inplace=True)
print('Created samples dataframe for manual actions entity')

Created samples dataframe for manual actions entity


In [9]:
samples.head(20)

,text,entity,label,start,end
0,2 . 1 . Os procedimentos para a realização de ...,informar,ACAO_MANUAL,2477,2485
1,2 . 1 . Os procedimentos para a realização de ...,informar,ACAO_MANUAL,3433,3441
2,2 . 1 . Os procedimentos para a realização de ...,energizar,ACAO_MANUAL,3351,3360
3,2 . 1 . Os procedimentos para a realização de ...,energizar,ACAO_MANUAL,3374,3383
4,2 . 1 . Os procedimentos para a realização de ...,desligamento,ACAO_MANUAL,495,507
5,2 . 1 . Os procedimentos para a realização de ...,desligamento,ACAO_MANUAL,1232,1244
6,2 . 1 . Os procedimentos para a realização de ...,desligamento,ACAO_MANUAL,1451,1463
7,2 . 1 . Os procedimentos para a realização de ...,desligamento,ACAO_MANUAL,1916,1928
8,2 . 1 . Os procedimentos para a realização de ...,desligamento,ACAO_MANUAL,2082,2094
9,2 . 1 . Os procedimentos para a realização de ...,desligamento,ACAO_MANUAL,4278,4290


In [20]:
text = df[df['Texto Limpo'].str.contains('capacitores de')]['Texto Limpo'].iloc[1]
text[text.index('capacitores de'):]
# text

'capacitores de 69 kV da SE Teresina desenergizados;- Bancos de capacitores de 13,8 kV da SE Piripiri desenergizados.Para energização simultânea dos transformadores T1 e T2, considerar as condições abaixo:- Tensão inferior ou igual a 238 kV;- Bancos de capacitores de 69 kV da SE Teresina desenergizados;- Bancos de capacitores de 13,8 kV da SE Piripiri desenergizados.(*) Caso um dos dois transformadores a serem energizados seja o 230/69 kV - 100 MVA - T7, após sua energização, separar as barras 04B1 e 04B2 e normalizar as cargas de 13,8 kV da SE Teresina.Observação: Em caso de dificuldade de controle de tensão do barramento 13,8 kV, deverão ser utilizados as fontes de reativo conectadas a este barramento.5COSR-NECOSR-NEEquatorial-PIProceder as possíveis transferências de cargas do setor 13,8 kV para o 69 kV, caso necessário.- Limite de carregamento dos transformadores 230 / 69 kV da SE Teresina, conforme CD-CT.NE.2OE.02. '

In [6]:
text2 = preproc.regex_subs_fragments(text)

In [7]:
# text2[1912:]

In [8]:
preproc.find_equipments_lt(text)

['LT 69 kV Boa Vista / Distrito com',
 'LT 69 kV Distrito / Floresta3.7.1',
 'LT 69 kV Distrito / Floresta',
 'LT 69 kV Distrito / Floresta3.7.2',
 'LT 230 kV UTE Jaguatirica II / Boa Vista',
 'LT 69 kV Boa Vista / Floresta C2',
 'LT 69 kV Boa Vista / Monte Cristo3',
 'LT 69 kV Jacitara / Boa Vista3.9.2',
 'LT 69 kV Boa Vista / Monte Cristo3.2.2',
 'LT 69 kV Floresta / Centro3.9',
 'LT 69 kV Serra da Lua / Mata Mata3.10',
 'LT 69 kV Boa Vista / Monte Cristo',
 'LT 69 kV Boa Vista / Floresta C23.5.1',
 'LT 69 kV Serra da Lua / Mata Mata3',
 'LT 69 kV Boa Vista / Floresta C2 ',
 'LT 69 kV Boa Vista / Distrito com derivação',
 'LT 69 kV Boa Vista / Floresta C1 com',
 'LT 69 kV Floresta / Centro3.8',
 'LT 69 kV Boa Vista / Monte Cristo3.2.1',
 'LT 69 kV Serra da Lua / Mata Mata4',
 'LT 69 kV Boa Vista / Centro',
 'LT 69 kV Boa Vista / Floresta C1',
 'LT 230 kV UTE Jaguatirica II / Boa Vista3',
 'LT 69 kV Floresta / Centro3.8.1',
 'LT 69 kV Boa Vista / Monte Cristo3.3',
 'LT 69 kV Distrito 

In [10]:
preproc.find_equipments_lt(text2)

['LT 69 kV Boa Vista / Distrito com',
 'LT 69 kV Jacitara / Boa Vista ',
 'LT 69 kV Distrito / Floresta',
 'LT 230 kV UTE Jaguatirica II / Boa Vista',
 'LT 69 kV Boa Vista / Floresta C2',
 'LT 69 kV Boa Vista / Monte Cristo',
 'LT 69 kV Boa Vista / Floresta C2 ',
 'LT 69 kV Boa Vista / Distrito com derivação',
 'LT 69 kV Boa Vista / Floresta C1 com',
 'LT 69 kV Boa Vista / Centro',
 'LT 69 kV Boa Vista / Floresta C1',
 'LT 69 kV Boa Vista / Monte Cristo 3',
 'LT 69 kV Boa Vista / Centro 3',
 'LT 69 kV Floresta / Centro',
 'LT 69 kV Jacitara / Boa',
 'LT 69 kV Distrito / Floresta 3',
 'LT 69 kV Boa Vista / Floresta C23',
 'LT 69 kV Serra da Lua / Mata Mata',
 'LT 69 kV Floresta / Centro 3',
 'LT 69 kV Serra da Lua / Mata MataA',
 'LT 69 kV Boa Vista / Distrito com derivação para',
 'LT 69 kV Serra da Lua / Mata MataA partir da',
 'LT 69 kV Serra da Lua / Mata MataA partir',
 'LT 69 kV Boa Vista / Floresta C1 com derivação',
 'LT 69 kV Boa Vista / Floresta C',
 'LT 230 kV UTE Jaguatirica

In [20]:
import regex as re
text = 'LT 69 kV Distrito / Floresta 3.7.1'
#text = 'Transformador 69/13,8 KV '
rule_list = [
    '[0-9]+\.[0-9]+.[0-9]+.[0-9]+'
    '[0-9]+\.[0-9]+.[0-9]+',
    '[0-9]+\.[0-9]+.[0-9]+',
    '[0-9]+\.[0-9]+. '
]
for rule in rule_list:
    text = re.sub(f'{rule}', '', text)
text

'LT 69 kV Distrito / Floresta '

In [11]:
import regex as re
re.sub(
    '[0-9]+\.[0-9]+\.[0-9]+',
    '',
    'LT 69 kV Boa Vista / Monte Cristo3.2.1'
)
#Transformador 230/69/13,8 kV 
# 'LT 230 kV UTE Jaguatirica II / Boa Vista3.1.1

'LT 69 kV Boa Vista / Monte Cristo'

In [10]:
df = wr.s3.read_parquet('s3://ons-ds-dev-entityrecognition/mpo/mvp/data/processed/operativestate_samples.parquet')

In [17]:
df.iloc[500:560].head(60)

,text,entity,label,start,end
500,2 . 1 . Os procedimentos para a realização de ...,energizado,ESTADO_OPERATIVO,4972,4982
501,2 . 1 . Os procedimentos para a realização de ...,energizado,ESTADO_OPERATIVO,5479,5489
502,2 . 1 . Os procedimentos para a realização de ...,energizado,ESTADO_OPERATIVO,6471,6481
503,2 . 1 . Os procedimentos para a realização de ...,ligado,ESTADO_OPERATIVO,5809,5815
504,2 . 1 . Os procedimentos para a realização de ...,ligado,ESTADO_OPERATIVO,9709,9715
505,2 . 1 . Os procedimentos para a realização de ...,ligado,ESTADO_OPERATIVO,9772,9778
506,2 . 1 . Os procedimentos para a realização de ...,ligado,ESTADO_OPERATIVO,9894,9900
507,2 . 1 . Os procedimentos para a realização de ...,ligado,ESTADO_OPERATIVO,10138,10144
508,2 . 1 . Os procedimentos para a realização de ...,ligado,ESTADO_OPERATIVO,10271,10277
509,2 . 1 . Os procedimentos para a realização de ...,ligado,ESTADO_OPERATIVO,10402,10408


In [34]:
substation_list = [
    '[0-9]+ kv',
    '[0-9]+,[0-9]+ kv',
    '[0-9]+ , [0-9]+ kv',
    '[0-9]+ kva',
    #'[0-9]+ kvb',
    #'[0-9]+ kvt',
    '[0-9]+ kw',
    '[0-9]+ mw',
    '[0-9]+ mvar',
    '[0-9]+ hz',
    '[0-9]+\,[0-9]+ hz',
    '[0-9]+ \ , [0-9]+ hz',
    '[0-9]+ ug',
    '[0-9]+ ugs',
    '[0-9]+ \( \w+ \) ugs',
    '[0-9]+°',
    '[0-9]+ °',
    '[0-9]+ graus',
    '[0-9]+,[0-9]+ graus'
    '[0-9]+ , [0-9]+ graus'
]

if 'nan' in substation_list:
    substation_list.remove('nan')
    
substation_list

['[0-9]+ kv',
 '[0-9]+,[0-9]+ kv',
 '[0-9]+ , [0-9]+ kv',
 '[0-9]+ kva',
 '[0-9]+ kw',
 '[0-9]+ mw',
 '[0-9]+ mvar',
 '[0-9]+ hz',
 '[0-9]+\\,[0-9]+ hz',
 '[0-9]+ \\ , [0-9]+ hz',
 '[0-9]+ ug',
 '[0-9]+ ugs',
 '[0-9]+ \\( \\w+ \\) ugs',
 '[0-9]+°',
 '[0-9]+ °',
 '[0-9]+ graus',
 '[0-9]+,[0-9]+ graus[0-9]+ , [0-9]+ graus']

In [35]:
def find_expressions_in_str(
    text:str,
    rule_list:list,
    exclude_rule_list=None,
    exclude_result_list=None
    ):

    import re

    expressions_list = []
    for rule in rule_list:
        try:
            expressions_list = expressions_list + re.findall(rule, text, re.IGNORECASE)
        except:
            expressions_list = expressions_list

    if exclude_rule_list != None:
        for rule in exclude_rule_list:
            expressions_list = [re.sub(rule, '', text) for text in expressions_list]

    if exclude_result_list != None:
        expressions_list = [i for i in expressions_list if i not in exclude_result_list]

    return expressions_list

In [37]:
find_expressions_in_str(
    '13 \ (palavra) \ ugs',
    substation_list
)

[]

In [5]:
preproc = preprocess_module()
print('Loaded preproc module')

df_fragments = preproc.get_fragments()
text = df_fragments[['Texto Limpo']].drop_duplicates()
text = preproc.preprocess_fragments(text, 'Texto Limpo')
print('Preprocessed fragments')

text = text[['processed']].copy()
text = preproc.find_valuewithunit(df=text, col='processed')
print('Found operative state in fragments\' text')

Loaded preproc module
Preprocessed fragments
Found operative state in fragments' text


In [6]:
text.head()

,processed,VALOR_COM_UNID.MEDIDA
0,Estabelecer procedimentos para o controle da t...,[]
1,2 . 1 . Os procedimentos para a realização de ...,[]
2,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,"[230 kV, 150 MVAR, 69 kV, 150 Mvar]"
3,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,"[230 kV, 150 MVAR, 69 kV, 150 Mvar]"
4,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,"[230 kV, 150 MVAR, 69 kV, 150 Mvar]"


In [7]:
samples = preproc.create_synthetic_samples(
    text,
    entity_col='VALOR_COM_UNID.MEDIDA',
    text_col='processed'
    )
samples.drop_duplicates(inplace=True)
print('Created samples dataframe for value with unit entity')

Created samples dataframe for value with unit entity


In [8]:
samples

,text,entity,label,start,end
0,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,230 kV,VALOR_COM_UNID.MEDIDA,101,107
1,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,230 kV,VALOR_COM_UNID.MEDIDA,160,166
2,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,230 kV,VALOR_COM_UNID.MEDIDA,238,244
3,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,150 MVAR,VALOR_COM_UNID.MEDIDA,2386,2394
4,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,69 kV,VALOR_COM_UNID.MEDIDA,287,292
...,...,...,...,...,...
83051,PassoCoordenaçãoControleComando/ExecuçãoProced...,500 kV,VALOR_COM_UNID.MEDIDA,338,344
83052,PassoCoordenaçãoControleComando/ExecuçãoProce...,230 kV,VALOR_COM_UNID.MEDIDA,239,245
83053,PassoCoordenaçãoControleComando/ExecuçãoProce...,230 kV,VALOR_COM_UNID.MEDIDA,385,391
83054,PassoCoordenaçãoControleComando/ExecuçãoProce...,80 Mvar,VALOR_COM_UNID.MEDIDA,454,461


In [9]:
bucket = 'ons-ds-dev-entityrecognition'
samples_path = f's3://{bucket}/mpo/mvp/data/processed/valuewithunit_samples.parquet'
samples.to_parquet(samples_path, compression='gzip')
print(f'saved samples in {samples_path}')

saved samples in s3://ons-ds-dev-entityrecognition/mpo/mvp/data/processed/valuewithunit_samples.parquet


In [ ]:
df = wr.s3.read_parquet('s3://ons-ds-dev-entityrecognition/mpo/mvp/data/processed/valuewithunit_samples.parquet')

In [ ]:
df.head()

In [5]:
df[df['text']==' Desenergização Iniciando pelo Terminal de 13 , 8 kV ( Sentido Único ) PassoCoordenaçãoControleComando / ExecuçãoProcedimentoObjetivo / Item de Controle 1O COSR-S deve solicitar à CEEE-D remanejar carga pela rede de distribuição da SE Porto Alegre 6 , pelo sistema de distribuição , para a SE Porto Alegre 12 ( até 9 MW ) , para a SE Rincão ( até 2 MW ) e para o transformador TR-3 da SE Porto Alegre 6 ( até 5 MW ) . 2COSR-SCOSR-SCEEE-TDesligar os alimentadores de 13 , 8 kV da SE Porto Alegre 6 , do barramento conectado ao Transformador TR-8 kV da SE Porto Alegre 6 . 3COSR-SCOSR-SCEEE-TDesligar o terminal de 13 , 8 kV do transformador TR-8 kV da SE Porto Alegre 6 . 4COSR-SCOSR-SCEEE-TDesenergizar , pelo terminal de 230 kV , o transformador TR-8 kV da SE Porto Alegre 6 . ']

,text,entity,label,start,end
28123,Desenergização Iniciando pelo Terminal de 13 ...,"13 , 8 kV",VALOR_COM_UNID.MEDIDA,43,52
28124,Desenergização Iniciando pelo Terminal de 13 ...,"13 , 8 kV",VALOR_COM_UNID.MEDIDA,466,475
28125,Desenergização Iniciando pelo Terminal de 13 ...,"13 , 8 kV",VALOR_COM_UNID.MEDIDA,613,622
28126,Desenergização Iniciando pelo Terminal de 13 ...,9 MW,VALOR_COM_UNID.MEDIDA,315,319
28127,Desenergização Iniciando pelo Terminal de 13 ...,8 kV,VALOR_COM_UNID.MEDIDA,48,52
28128,Desenergização Iniciando pelo Terminal de 13 ...,8 kV,VALOR_COM_UNID.MEDIDA,471,475
28129,Desenergização Iniciando pelo Terminal de 13 ...,8 kV,VALOR_COM_UNID.MEDIDA,543,547
28130,Desenergização Iniciando pelo Terminal de 13 ...,8 kV,VALOR_COM_UNID.MEDIDA,618,622
28131,Desenergização Iniciando pelo Terminal de 13 ...,8 kV,VALOR_COM_UNID.MEDIDA,643,647
28132,Desenergização Iniciando pelo Terminal de 13 ...,8 kV,VALOR_COM_UNID.MEDIDA,750,754


In [4]:
df.iloc[21178]

text       1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...
entity                                               230 kV
label                                 VALOR_COM_UNID.MEDIDA
start                                                  3380
end                                                    3386
Name: 21180, dtype: object

In [29]:
df['text'].iloc[21178]

' Desenergização Iniciando pelo Terminal de 13 , 8 kV ( Sentido Único ) PassoCoordenaçãoControleComando / ExecuçãoProcedimentoObjetivo / Item de Controle 1O COSR-S deve solicitar à CEEE-D remanejar carga pela rede de distribuição da SE Porto Alegre 6 , pelo sistema de distribuição , para a SE Porto Alegre 12 ( até 9 MW ) , para a SE Rincão ( até 2 MW ) e para o transformador TR-3 da SE Porto Alegre 6 ( até 5 MW ) . 2COSR-SCOSR-SCEEE-TDesligar os alimentadores de 13 , 8 kV da SE Porto Alegre 6 , do barramento conectado ao Transformador TR-8 kV da SE Porto Alegre 6 . 3COSR-SCOSR-SCEEE-TDesligar o terminal de 13 , 8 kV do transformador TR-8 kV da SE Porto Alegre 6 . 4COSR-SCOSR-SCEEE-TDesenergizar , pelo terminal de 230 kV , o transformador TR-8 kV da SE Porto Alegre 6 . '

In [8]:
df[df['text'].str.contains('linha de transmissão') & df['entity'].str.contains('linha de transmissão')]['text'].iloc[0][783:]

'linhas de transmissão e transformadores da Rede de Operação do Sistema Roraima . Para linhas de transmissão e transformadores , pertencentes à Rede de Operação e que são de uso exclusivo da geração ou de funções transmissão de controle de reativos , bem como para os demais equipamentos pertencentes a Rede de Operação , só constam aqueles cujas manobras necessitam de procedimentos específicos de ajustes no sistema . 2 . 4 . Os procedimentos para desligamento programado ou de urgência de linhas de transmissão e de equipamentos são sempre coordenados pelo ONS . Quando se tratar de manobras programadas , os procedimentos contidos nas recomendações das análises do desligamento prevalecem sobre os procedimentos contidos nesta Instrução de Operação . 2 . 5 . Os procedimentos para realização de manobras de desenergização devem atender os limites operativos sistêmicos definidos pelo ONS e os limites operativos declarados pelos agentes nos Cadastros de Informações Operacionais de Limites de Equ

In [6]:
import regex as re
result = re.search(
    'ltc na posição [0-9]+(\w+)',
    'ltc na posição 13b',
)
result[0]

,text,entity,label,start,end
60,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,energizado,ESTADO_OPERATIVO,9505,9515
61,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,energizado,ESTADO_OPERATIVO,10480,10490
62,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,ligado,ESTADO_OPERATIVO,9835,9841
63,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,ligada,ESTADO_OPERATIVO,10117,10123
64,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,desenergizada,ESTADO_OPERATIVO,8700,8713
65,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,sob carga,ESTADO_OPERATIVO,9558,9567
66,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,sob carga,ESTADO_OPERATIVO,9774,9783
67,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,desligada,ESTADO_OPERATIVO,10114,10123
68,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,aberto,ESTADO_OPERATIVO,8558,8564
69,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...,desligadas,ESTADO_OPERATIVO,10114,10124


In [10]:
df = wr.s3.read_parquet('s3://ons-ds-dev-entityrecognition/mpo/mvp/data/raw/marked_samples/poc_marked.parquet')
df.head()

,region,text,start,end,label
0,cosr-nco,PassoCoordenaçãoControleComando / ExecuçãoProc...,97,106,ACAO_A_EXECUTAR
1,cosr-nco,PassoCoordenaçãoControleComando / ExecuçãoProc...,270,279,ACAO_A_EXECUTAR
2,cosr-nco,PassoCoordenaçãoControleComando / ExecuçãoProc...,373,381,ACAO_A_EXECUTAR
3,cosr-nco,PassoCoordenaçãoControleComando / ExecuçãoProc...,460,474,ACAO_A_EXECUTAR
4,cosr-nco,PassoCoordenaçãoControleComando / ExecuçãoProc...,493,502,ACAO_A_EXECUTAR


In [12]:
df['label'].value_counts()

ACAO_A_EXECUTAR          6974
EQUIPAMENTO              4992
VALOR_COM_UNID.MEDIDA    4864
SUBESTACAO               4388
OPERADOR_MATEMATICO      2346
INEQUACAO                2220
ESTADO_OPERATIVO         1408
USINA                     872
Name: label, dtype: Int64

In [14]:
df = df[df['label'] == 'SUBESTACAO']
df.head()

,region,text,start,end,label
9,cosr-nco,PassoCoordenaçãoControleComando / ExecuçãoProc...,475,492,SUBESTACAO
19,cnos,PassoCoordenaçãoControleComando/ ExecuçãoProce...,355,378,SUBESTACAO
21,cnos,PassoCoordenaçãoControleComando/ ExecuçãoProce...,379,393,SUBESTACAO
27,cnos,PassoCoordenaçãoControleComando/ ExecuçãoProce...,481,495,SUBESTACAO
35,cnos,PassoCoordenaçãoControleComando / execuçãoProc...,479,490,SUBESTACAO


In [15]:
def create_entity_col(x):
    text = x['text']
    start = x['start']
    end = x['end']
    return text[start-1:end]

In [16]:
df['entity'] = df[['text','start','end']].apply(lambda x:create_entity_col(x), axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
df.tail(60)

,region,text,start,end,label,entity
27674,cnos-equipnull,PassoCoordenaçãoControleComando / ExecuçãoProc...,697,707,SUBESTACAO,SE Sapeaçu.
27675,cnos-equipnull,PassoCoordenaçãoControleComando / ExecuçãoProc...,787,797,SUBESTACAO,SE Sapeaçu
27686,cosr-nco,3.13.2.1. Energização da LT 230 kV São Luís I...,86,99,SUBESTACAO,SE Miranda II
27690,cosr-nco,3.13.2.1. Energização da LT 230 kV São Luís I...,236,249,SUBESTACAO,SE Miranda II
27695,cosr-nco,3.13.2.1. Energização da LT 230 kV São Luís I...,405,418,SUBESTACAO,SE Miranda II.
27697,cosr-nco,3.13.2.1. Energização da LT 230 kV São Luís I...,419,432,SUBESTACAO,SE Miranda II
27702,cosr-nco,3.13.2.1. Energização da LT 230 kV São Luís I...,528,542,SUBESTACAO,SE São Luís II.
27710,cosr-nco,3.13.2.1. Energização da LT 230 kV São Luís I...,646,660,SUBESTACAO,SE São Luís II
27715,cosr-nco,3.13.2.1. Energização da LT 230 kV São Luís I...,858,872,SUBESTACAO,SE São Luís II.
27717,cosr-nco,3.13.2.1. Energização da LT 230 kV São Luís I...,873,887,SUBESTACAO,SE São Luís II


In [58]:
df_syn = wr.s3.read_parquet('s3://ons-ds-dev-entityrecognition/mpo/mvp/data/processed/substation_samples.parquet')

In [66]:
df_syn[df_syn['label'].str.contains('se bateiasi')]

,text,label,start,end
8067,energizacao iniciando pela se bateias ( senti...,se bateiasigual ou inferior a,496,525
8076,energizacao iniciando pela se bateias ( senti...,se bateiasigual ou inferior a,486,515
8083,energizacao iniciando pela se bateias ( senti...,se bateiasigual ou inferior a,490,519
8094,energizacao iniciando pela se bateias ( senti...,se bateiasigual ou inferior a,498,527
16768,energizacao iniciando pela se campo comprido ...,se bateiasigual ou inferior a,1717,1746
16957,energizacao iniciando pelo lado 230 kv ( sent...,se bateiasigual ou inferior a,526,555


In [8]:
preproc = preprocess_module()
print('Loaded preproc module')

df_fragments = preproc.get_fragments()
text = df_fragments[['Texto Limpo']].drop_duplicates()
text = preproc.preprocess_fragments(text, 'Texto Limpo')

Loaded preproc module


In [9]:
text

,Texto Limpo,processed
0,estabelecer procedimentos para o controle da t...,estabelecer procedimentos para o controle da t...
1,2.1. os procedimentos para a reali...,2 . 1 . os procedimentos para a realizacao de ...
2,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...
3,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...
4,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...


In [3]:
df = wr.s3.read_parquet('s3://ons-ds-dev-entityrecognition/mpo/mvp/data/processed/substation_samples.parquet')

In [6]:
df.dropna().head(20)

,text,label,start,end
0,1 . . preparacao para manobras em linhas de t...,se equatorial,445,458
1,1 . . preparacao para manobras em linhas de t...,se equatorial,563,576
2,1 . . preparacao para manobras em linhas de t...,se equatorial,678,691
3,1 . . preparacao para manobras em linhas de t...,se equatorial,2825,2838
4,1 . . preparacao para manobras em linhas de t...,se equatorial,2920,2933
5,1 . . preparacao para manobras em linhas de t...,se equatorial,2984,2997
7,1 . . preparacao para manobras em linhas de t...,se equatorial,445,459
8,1 . . preparacao para manobras em linhas de t...,se equatorial,563,577
9,1 . . preparacao para manobras em linhas de t...,se equatorial,678,692
10,1 . . preparacao para manobras em linhas de t...,se equatorial,2825,2839


In [5]:
df['label'].value_counts(normalize=True).iloc[0:60].head(60)

se anastacio                  0.018734
se presidente                 0.015212
se presidente dutra           0.010911
se corumba 2                  0.009381
se campo grande 2              0.00892
se sao                        0.007649
se araraquara 2               0.007534
se rio                         0.00739
se boa                        0.007216
se araraquara 2               0.006437
se equatorial                 0.006091
se equatorial                 0.005946
se nova                       0.005889
se elevadora paulino neves    0.005831
se elevadora paulino          0.005773
se elevadora                  0.005773
se boa esperanca              0.005571
se ilha so lteira 2           0.005542
se porto                       0.00534
se pocos de caldas            0.005254
se bom                        0.004936
se foz do                     0.004676
se pocos de                   0.004647
se janauba 3                   0.00459
se campos                     0.004503
se santa                 

In [6]:
df = wr.s3.read_parquet('s3://ons-ds-dev-entityrecognition/mpo/mvp/data/processed/powerplant_samples.parquet')

In [7]:
df

,text,label,start,end
0,1 . . preparacao para manobras em linhas de t...,ute monte cristo bloco 2,701,725
1,1 . . preparacao para manobras em linhas de t...,ute monte cristo ii,3223,3242
2,1 . . preparacao para manobras em linhas de t...,ute monte cristo ii,3330,3349
3,1 . . preparacao para manobras em linhas de t...,ute monte cristo ii,3401,3420
4,1 . . preparacao para manobras em linhas de t...,ute jaguatirica i,75,92
...,...,...,...,...
11382,1 . . preparacao para manobras em linhas de t...,uhe santo antonio,6733,6750
11383,1 . . preparacao para manobras em linhas de t...,uhe santo antonio,6880,6897
11384,1 . . preparacao para manobras em linhas de t...,uhe santo antonio,6979,6996
11385,1 . . preparacao para manobras em linhas de t...,uhe belo monte,23868,23882


In [10]:
df['label'].value_counts()[120:180]

ufv sjp energizacao                    20
uhe mascarenhas de moraes 4            19
uhe paulo afonsosentido do             19
uhe corumba                            19
ute monte cristo                       18
ute juiz de fora                       18
uhe quebra queixo                      18
uhe quebra                             18
uhe nova                               18
uhe belo monte                         18
ute monte                              18
ute juiz de                            18
ute termonorte ii                      17
uhe passo sao joao                     16
ute palmeiras de goias                 16
ute palmeiras                          16
ute palmeiras de                       16
pch cachoeirao                         16
ute canoas                             16
uhe sao simao 5                        16
ute pampa                              15
uhe machadinho                         15
uhe peixe angical autotransformador    15
uhe barra grande                  

In [3]:
preproc = preprocess_module()
print('Loaded preproc module')

df_fragments = preproc.get_fragments()
text = df_fragments[['Texto Limpo']].drop_duplicates()
text = preproc.preprocess_fragments(text, 'Texto Limpo')
print('Preprocessed fragments')

Loaded preproc module
Preprocessed fragments


In [5]:
text = text[['processed']].copy()
text = preproc.find_mathoperator(df=text, col='processed')
text = text.astype('str').drop_duplicates()
print('Found math operator in fragments\' text')

Found math operator in fragments' text


In [6]:
samples = preproc.create_synthetic_samples(
    text,
    entity_col='OPERADOR_MATEMATICO',
    text_col='processed'
    )
print('Created samples dataframe for math operator entity')

samples.head()

KeyboardInterrupt: 

In [7]:
text.head()

,processed,OPERADOR_MATEMATICO
0,estabelecer procedimentos para o controle da t...,[]
1,2 . 1 . os procedimentos para a realizacao de ...,"[superior, entre, inferior]"
2,1 . . preparacao para manobras em linhas de t...,"[superior, entre, inferior]"
3,1 . . preparacao para manobras em linhas de t...,"[superior, entre, inferior]"
4,1 . . preparacao para manobras em linhas de t...,"[superior, entre, inferior]"


In [5]:
text = text[['processed']].copy()
text = preproc.find_mathoperator(df=text, col='processed')
print('Found math operator in fragments\' text')

samples = preproc.create_synthetic_samples(
    text,
    entity_col='OPERADOR_MATEMATICO',
    text_col='processed'
    )
print('Created samples dataframe for math operator entity')

bucket = 'ons-ds-dev-entityrecognition'
samples_path = f's3://{bucket}/mpo/mvp/data/processed/mathoperator_samples.parquet'
samples.to_parquet(samples_path, compression='gzip')
print(f'saved samples in {samples_path}')

,Texto Limpo,processed
0,estabelecer procedimentos para o controle da t...,estabelecer procedimentos para o controle da t...
1,2.1. os procedimentos para a reali...,2 . 1 . os procedimentos para a realizacao de ...
2,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...
3,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...
4,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...
...,...,...
7272,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...
7273,3.21.2.1. energização (ambos os sentidos)pass...,energizacao ( ambos os sentidos ) e item de c...
7274,3.6.2.1. energização iniciando pela se ba...,energizacao iniciando pela se banabuiu ( sent...
7275,passocoordenaçãocontrolecomando/execuçãoproced...,/objetivo / item de controle 1para desenergiza...


In [7]:
df_2 = wr.s3.read_parquet('s3://ons-ds-dev-entityrecognition/mpo/mvp/data/processed/valuewithunit_samples.parquet')
df_2.head(20)

,text,label,start,end
0,PassoCoordenaçãoControleComando / ExecuçãoProc...,230 kV,211,217
1,PassoCoordenaçãoControleComando / ExecuçãoProc...,230 kV,386,392
2,PassoCoordenaçãoControleComando/ ExecuçãoProce...,230 kV,179,185
3,PassoCoordenaçãoControleComando/ ExecuçãoProce...,230 kV,308,314
4,PassoCoordenaçãoControleComando/ ExecuçãoProce...,230 kV,420,426
5,PassoCoordenaçãoControleComando / execuçãoProc...,525 kV,208,214
6,PassoCoordenaçãoControleComando / execuçãoProc...,525 kV,396,402
7,PassoCoordenaçãoControleComando / execuçãoProc...,525 kV,536,542
8,3.33.2.1. Energização da LT 230 kV Cascavel O...,"13,8 kV",704,711
9,3.33.2.1. Energização da LT 230 kV Cascavel O...,230 kV,29,35


In [16]:
'ABC'.lower()

'abc'

In [4]:
bucket = 'ons-dl-dev-landing'
file = 'ons/mpo_ds_nlp/data/preprocessed_output/spacy.parquet'
obj_uri = 's3://{}/{}'.format(bucket, file)
df = wr.s3.read_parquet(obj_uri)
df

,region,text,start,end,label
0,cosr-nco,PassoCoordenaçãoControleComando / ExecuçãoProc...,97,106,ACAO_A_EXECUTAR
1,cosr-nco,PassoCoordenaçãoControleComando / ExecuçãoProc...,270,279,ACAO_A_EXECUTAR
2,cosr-nco,PassoCoordenaçãoControleComando / ExecuçãoProc...,373,381,ACAO_A_EXECUTAR
3,cosr-nco,PassoCoordenaçãoControleComando / ExecuçãoProc...,460,474,ACAO_A_EXECUTAR
4,cosr-nco,PassoCoordenaçãoControleComando / ExecuçãoProc...,493,502,ACAO_A_EXECUTAR
...,...,...,...,...,...
28059,cosr-se,PassoCoordenaçãoControleComando / ExecuçãoProc...,369,390,INEQUACAO
28060,cosr-se,PassoCoordenaçãoControleComando / ExecuçãoProc...,369,375,VALOR_COM_UNID.MEDIDA
28061,cosr-se,PassoCoordenaçãoControleComando / ExecuçãoProc...,376,377,OPERADOR_MATEMATICO
28062,cosr-se,PassoCoordenaçãoControleComando / ExecuçãoProc...,378,379,VALOR_COM_UNID.MEDIDA


In [5]:
df['label'].value_counts()

ACAO_A_EXECUTAR          6974
EQUIPAMENTO              4992
VALOR_COM_UNID.MEDIDA    4864
SUBESTACAO               4388
OPERADOR_MATEMATICO      2346
INEQUACAO                2220
ESTADO_OPERATIVO         1408
USINA                     872
Name: label, dtype: Int64

In [6]:
# equipment, manualactions, operativestate, valuewithunit
# mathoperator, substation

In [7]:
def create_entity_col(x):
    text = x['text']
    start = x['start']
    end = x['end']
    return text[start-1:end]

In [8]:
df['entity'] = df[['text','start','end']].apply(lambda x:create_entity_col(x), axis=1)

In [9]:
df_fragments = df[['text']].copy()
df_fragments = df_fragments.drop_duplicates()
df_fragments.head()

,text
0,PassoCoordenaçãoControleComando / ExecuçãoProc...
10,PassoCoordenaçãoControleComando/ ExecuçãoProce...
15,PassoCoordenaçãoControleComando/ ExecuçãoProce...
28,PassoCoordenaçãoControleComando / execuçãoProc...
37,3.33.2.1. Energização da LT 230 kV Cascavel O...


In [10]:
df = preproc.find_mathoperator(
    df_fragments,
    col='text'
    )
df.head()

,text,OPERADOR_MATEMATICO
0,PassoCoordenaçãoControleComando / ExecuçãoProc...,[]
10,PassoCoordenaçãoControleComando/ ExecuçãoProce...,[]
15,PassoCoordenaçãoControleComando/ ExecuçãoProce...,[≤]
28,PassoCoordenaçãoControleComando / execuçãoProc...,[]
37,3.33.2.1. Energização da LT 230 kV Cascavel O...,"[inferior, igual ou inferior]"


In [11]:
samples_vu = preproc.create_synthetic_samples(
    df_fragments,
    entity_col='VALOR_COM_UNID.MEDIDA',
    text_col='text'
    )
samples_vu.head()

,text,label,start,end
0,PassoCoordenaçãoControleComando / ExecuçãoProc...,230 kV,211,217
1,PassoCoordenaçãoControleComando / ExecuçãoProc...,230 kV,386,392
2,PassoCoordenaçãoControleComando/ ExecuçãoProce...,230 kV,179,185
3,PassoCoordenaçãoControleComando/ ExecuçãoProce...,230 kV,308,314
4,PassoCoordenaçãoControleComando/ ExecuçãoProce...,230 kV,420,426


In [12]:
bucket = 'ons-ds-dev-entityrecognition'
samples_vu_path = f's3://{bucket}/mpo/mvp/data/processed/valuewithunit_samples.parquet'
samples_vu.to_parquet(samples_vu_path, compression='gzip')
print(f'saved samples in {samples_vu_path}')

saved samples in s3://ons-ds-dev-entityrecognition/mpo/mvp/data/processed/valuewithunit_samples.parquet


In [10]:
df_eo = df[df['label']=='ESTADO_OPERATIVO']

In [29]:
operative_state_list = [
    #'Reator de linha da SE (\w+) (\w+) (\w+) conectado',
    #'Reator de linha da SE (\w+) (\w+) conectado',
    #'Reator de linha da SE (\w+) conectado',
    'condições especificadas na (\w+)-(\w+).(\w+).(\w+)',
    'indisponível',
    'indisponíveis',
    'indisponibilidade',
    'LTC na posição 13b',
    'com indisponibilidade',
    'fluxo de potência ativa',
    'indisponibilidade simples',
    'fluxo nulo no transformador',
    'independente das condições operativas da UTE',
    'ligado',
    'ligada',
    'fechado',
    'fechada',
    'fechadas',
    'fechados',
    'desligado',
    'desligada',
    'desligados',
    'desligadas',
    'conectado',
    'conectada',
    'conectados',
    'conectadas',
    'em operação',
    'desconectado',
    'desconectada',
    'desconectados',
    'desconectadas',
    'sincronizada',
    'sincronizado',
    'sincronizados',
    'sincronizadas',
    'fora de operação',
    'fora da operação',
    'conectado para ser energizado',
    'carga ativa em pelo menos um dos circuitos',
    'restrição de regime permanente',
    'interligado',
    'interligada',
    'interligados',
    'interligadas',
    'inserido',
    'inserida',
    'baipassado',
    'energizado em vazio',
    'em operação automática',
    'sobrecarga',
    'sobrecarregado',
    'sobrecarregada',
    'sobrecarregados',
    'sobrecarregadas',
    'aberto',
    'aberta',
    'abertos',
    'abertas',
    'sem carga',
    'em carga',
    'em vazio',
    'fluxo nulo',
    'com folga',
    'em série',
    'sob carga',
    'modo automático',
    'em carregamento',
    'em barramento',
    'em um barramento',
    'energizado',
    'energizada',
    'energizados',
    'energizadas',
    'desenergizado',
    'desenergizada',
    'desenergizados',
    'desenergizados',
    'opere sozinha em um barramento',
    'energizado juntamente com a LTB',
    'energizado em pelo menos um circuito',
    'ajustado',
    'ajustada',
    'ajustados',
    'ajustadas',
    'bloqueado',
    'bloqueada',
    'bloqueados',
    'bloqueadas',
    'disponível para ser sincronizada',
    'disponível para ser sincronizado'
]

In [30]:
operative_state_list

['condições especificadas na (\\w+)-(\\w+).(\\w+).(\\w+)',
 'indisponível',
 'indisponíveis',
 'indisponibilidade',
 'LTC na posição 13b',
 'com indisponibilidade',
 'fluxo de potência ativa',
 'indisponibilidade simples',
 'fluxo nulo no transformador',
 'independente das condições operativas da UTE',
 'ligado',
 'ligada',
 'fechado',
 'fechada',
 'fechadas',
 'fechados',
 'desligado',
 'desligada',
 'desligados',
 'desligadas',
 'conectado',
 'conectada',
 'conectados',
 'conectadas',
 'em operação',
 'desconectado',
 'desconectada',
 'desconectados',
 'desconectadas',
 'sincronizada',
 'sincronizado',
 'sincronizados',
 'sincronizadas',
 'fora de operação',
 'fora da operação',
 'conectado para ser energizado',
 'carga ativa em pelo menos um dos circuitos',
 'restrição de regime permanente',
 'interligado',
 'interligada',
 'interligados',
 'interligadas',
 'inserido',
 'inserida',
 'baipassado',
 'energizado em vazio',
 'em operação automática',
 'sobrecarga',
 'sobrecarregado',
 '

In [2]:
import regex as re

In [25]:
result = re.search(
    'Fluxo (\w+) no circuito energizado', 
    'Fluxo (MW) no circuito energizado'
)

In [26]:
result[0]

TypeError: 'NoneType' object is not subscriptable

---
## Reading input data

In [4]:
df_fragments = preproc.get_fragments()
df_fragments.head()

,MpoCentro,Texto Limpo
0,cosr-nco,estabelecer procedimentos para o controle da t...
1,cosr-nco,2.1. os procedimentos para a reali...
2,cosr-nco,1. objetivo2....................................
3,cosr-nco,1. objetivo2....................................
4,cosr-nco,1. objetivo2....................................


In [5]:
# df_equipments = preproc.get_equipments()
# df_equipments.head()

In [6]:
# df_equipments_fameq = preproc.get_equipments_fameq()
# df_equipments_fameq.head()

There is no interesction between guid_tipoequipamentosgi and guid_equipamento:

In [7]:
# pd.merge(left=df_equipments, right=df_equipments_fameq, left_on='guid_equipamento', right_on='guid_tipoequipamentosgi', how='inner')

---
## Preprocessing datasets

In [8]:
df_fragments = preproc.preprocess_fragments(df_fragments, 'Texto Limpo')
df_fragments.head()

,MpoCentro,Texto Limpo,processed
0,cosr-nco,estabelecer procedimentos para o controle da t...,estabelecer procedimentos para o controle da t...
1,cosr-nco,2.1. os procedimentos para a reali...,2 . 1 . os procedimentos para a realizacao de ...
2,cosr-nco,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...
3,cosr-nco,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...
4,cosr-nco,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...


In [9]:
# df_equipments = preproc.preprocess_equipments(df_equipments, 'nom_longoequipamento')
# df_equipments.head()

---

## Marking input data

### Marking equipments

In [10]:
df_fragments_eq = preproc.find_equipments(df=df_fragments, col='processed')
df_fragments_eq.astype('str').drop_duplicates(subset=['MpoCentro', 'processed', 'EQUIPAMENTO'], keep='last', inplace=True)
df_fragments_eq.head()

,MpoCentro,Texto Limpo,processed,EQUIPAMENTO
0,cosr-nco,estabelecer procedimentos para o controle da t...,estabelecer procedimentos para o controle da t...,[]
1,cosr-nco,2.1. os procedimentos para a reali...,2 . 1 . os procedimentos para a realizacao de ...,[]
2,cosr-nco,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...,"[lt 69 kv boa vista / monte cristo, lt 230 kv ..."
3,cosr-nco,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...,"[lt 69 kv boa vista / monte cristo, lt 230 kv ..."
4,cosr-nco,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...,"[lt 69 kv boa vista / monte cristo, lt 230 kv ..."


In [10]:
samples = preproc.create_synthetic_samples(
    df_fragments_eq,
    entity_col='EQUIPAMENTO',
    text_col='processed'
    )

In [ ]:
samples.head()

In [ ]:
bucket = 'ons-ds-dev-entityrecognition'
samples_equipments_path = f's3://{bucket}/mpo/mvp/data/processed/equipment_samples.parquet'
samples.to_parquet(samples_equipments_path, compression='gzip')

### Marking manual actions

In [11]:
df_manualactions = preproc.get_manual_actions(
    bucket='ons-ds-dev-entityrecognition',
    path='mpo/mvp/data/raw/manual_actions'
    )

df_manualactions.head()

,termos,actions
0,Limitações impostas pela (nome da IO-OC) atend...,limitações impostas pela
1,Atender previamente as limitações para a indis...,atender previamente as limitações
2,Adotar os procedimentos para a indisponibilidade,adotar os procedimentos para a indisponibilidade
3,Limitações impostas pela indisponibilidade da(...,limitações impostas pela indisponibilidade
4,Verificar se o somatório dos fluxos nos circui...,verificar o somatório dos fluxos


In [12]:
df_manualactions = preproc.preprocess_fragments(df_manualactions, 'actions')
df_manualactions.head()

,termos,actions,processed
0,Limitações impostas pela (nome da IO-OC) atend...,limitações impostas pela,limitacoes impostas pela
1,Atender previamente as limitações para a indis...,atender previamente as limitações,atender previamente as limitacoes
2,Adotar os procedimentos para a indisponibilidade,adotar os procedimentos para a indisponibilidade,adotar os procedimentos para a indisponibilidade
3,Limitações impostas pela indisponibilidade da(...,limitações impostas pela indisponibilidade,limitacoes impostas pela indisponibilidade
4,Verificar se o somatório dos fluxos nos circui...,verificar o somatório dos fluxos,verificar o somatorio dos fluxos


In [ ]:
df_fragments_ma = preproc.find_manual_actions(
    df_fragments,
    df_manualactions
    )

df_fragments_ma.head()

---

### Create synthetic samples

In [ ]:
df_fragments_ma.astype('str').drop_duplicates(subset=['MpoCentro', 'processed', 'ACAOMANUAL'], keep='last', inplace=True)
samples_manualactions = preproc.create_synthetic_samples(
    df_fragments_ma,
    entity_col='ACAOMANUAL',
    text_col='processed'
    )
samples_manualactions.head()

In [ ]:
bucket = 'ons-ds-dev-entityrecognition'
samples_manualactions_path = f's3://{bucket}/mpo/mvp/data/processed/manualactions_samples.parquet'
samples_manualactions.to_parquet(samples_manualactions_path, compression='gzip')